In [1]:
import pandas as pd

In [2]:
main = pd.read_csv('dice_com-job_us_sample.csv')

In [3]:
main.head()

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,Please see job description,3941b2f206ae0f900c4fba4ac0b18719
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"Configuration Management, Developer, Linux, Ma...",45efa1f6bc65acc32bbbb953a1ed13b7


In [4]:
main = main[['company', 'employmenttype_jobstatus', 'joblocation_address', 'jobtitle', 'jobdescription', 'shift', 'skills']]

In [5]:
main = main.rename(columns={"jobtitle": "job_title", "joblocation_address": "state"})

Change State as state abrev

In [6]:
main['state'] = main['state'].str.split(', ').str[1].str[:2].str.upper()

Remove rows that are NA or include 'see' under skills

In [7]:
main = main[~main['skills'].str.lower().str.contains('see', na=False) & main['skills'].notna() & ~main['skills'].str.lower().str.contains('null', na=False)]

Remote=1 if remote is mentioned in job title or job status, or is shift says 'telecommuting available'

In [8]:
main['remote'] = (main['job_title'].str.lower().str.contains('remote', na=False)) | (main['employmenttype_jobstatus'].str.lower().str.contains('remote', na=False)) | main['shift'].str.lower().str.contains('telecommuting available', na=False)

In [9]:
main['remote'] = main['remote'].astype(int)

In [10]:
main.head()

,company,employmenttype_jobstatus,state,job_title,jobdescription,shift,skills,remote
1,University of Chicago/IT Services,Full Time,IL,Information Security Engineer,The University of Chicago has a rapidly growin...,Telecommuting not available|Travel not required,"linux/unix, network monitoring, incident respo...",0
2,"Galaxy Systems, Inc.",Full Time,IL,Business Solutions Architect,"GalaxE.SolutionsEvery day, our solutions affec...",Telecommuting not available|Travel not required,"Enterprise Solutions Architecture, business in...",0
4,Matrix Resources,Full Time,GA,DevOps Engineer,Midtown based high tech firm has an immediate ...,Telecommuting not available|Travel not required,"Configuration Management, Developer, Linux, Ma...",0
5,Yash Technologies,"Full Time, Permanant",IL,SAP FICO Architect,We are looking for a Senior SAP FICO Architect...,Telecommuting not available|Travel required to...,"FICO, AR, AP, Asset Management, HAHA",0
6,Noble1,"Full Time, Direct Hire",GA,Network Engineer,Network Engineer Job Description A Network Eng...,Telecommuting not available|Travel not required,"Cisco, DNS, HTTP, Networking, Network Engineer...",0


Full Time=1 if 'full time' is mentioned in job title or job status

In [11]:
main['full_time'] = (main['employmenttype_jobstatus'].str.lower().str.contains('full time', na=False)) | (main['job_title'].str.lower().str.contains('full time', na=False))
main['full_time'] = main['full_time'].astype(int)

Contract=1 if 'contract' is mentioned in job title or job status

In [12]:
main['contract'] = (main['employmenttype_jobstatus'].str.lower().str.contains('contract', na=False)) | (main['job_title'].str.lower().str.contains('contract', na=False))
main['contract'] = main['contract'].astype(int)

Other=1 if neither 'full time' or 'contract' are mentioned in job title or job status

In [13]:
main['other'] = (~main['employmenttype_jobstatus'].str.lower().str.contains('contract', na=False)) & (~main['employmenttype_jobstatus'].str.lower().str.contains('full time', na=False)) & (~main['job_title'].str.lower().str.contains('contract', na=False)) & (~main['job_title'].str.lower().str.contains('full time', na=False))
main['other'] = main['other'].astype(int)

In [14]:
main.drop(columns=['employmenttype_jobstatus', 'shift'])

,company,state,job_title,jobdescription,skills,remote,full_time,contract,other
1,University of Chicago/IT Services,IL,Information Security Engineer,The University of Chicago has a rapidly growin...,"linux/unix, network monitoring, incident respo...",0,1,0,0
2,"Galaxy Systems, Inc.",IL,Business Solutions Architect,"GalaxE.SolutionsEvery day, our solutions affec...","Enterprise Solutions Architecture, business in...",0,1,0,0
4,Matrix Resources,GA,DevOps Engineer,Midtown based high tech firm has an immediate ...,"Configuration Management, Developer, Linux, Ma...",0,1,0,0
5,Yash Technologies,IL,SAP FICO Architect,We are looking for a Senior SAP FICO Architect...,"FICO, AR, AP, Asset Management, HAHA",0,1,0,0
6,Noble1,GA,Network Engineer,Network Engineer Job Description A Network Eng...,"Cisco, DNS, HTTP, Networking, Network Engineer...",0,1,0,0
...,...,...,...,...,...,...,...,...,...
21994,SNW Infotech,CA,Empower Administrator,"• Hands on experience in CSV, Performing I...","Empower3, LIMPS",0,0,1,0
21995,IAC Publishing,CA,Web Designer,Company Description We are searching for a ta...,"UI/UX mobile apps, interaction design, digital...",0,1,0,0
21996,Omega Solutions Inc,CA,Senior Front End Web Developer - Full Time at ...,CONTACT - priya@omegasolutioninc.com / 408-45...,"JavaScript, HTML5, CSS3, Bootstrap, AJAX, Reac...",0,1,0,0
21997,San Francisco Health Plan,CA,QA Analyst,Do you take pride in your work knowing that th...,"SDLC, ALM, SQL, T-SQL, RedGate, Team Foundatio...",0,1,0,0


In [15]:
main['year'] = 2017

only include jobs that have 'data' or 'analyst' in the job title

In [16]:
main = main[main['job_title'].str.lower().str.contains('data', na=False) | main['job_title'].str.lower().str.contains('analyst', na=False)]

In [17]:
main.head()

,company,employmenttype_jobstatus,state,job_title,jobdescription,shift,skills,remote,full_time,contract,other,year
11,CSI (Consultant Specialists Inc.),"Full Time, Contract Corp-To-Corp, Contract Ind...",CO,9001 Data Security Administrator - Unix & IAM,"Must have skills: UNIX, IAM (Identity Access M...",Telecommuting not available|Travel not required,"Unix, IAM, Scripting knowledge, OIM, Windows, ...",0,1,1,0,2017
17,"VanderHouwen & Associates, Inc.","Full Time, Full time",OR,Principal Application Analyst-Supply Chain Lawson,VanderHouwen has more jobs you may like! Find...,Telecommuting not available|Travel not required,Lawson Supply Chain,0,1,0,0,2017
20,Genesis10,Full Time,GA,IT Business Analyst,Genesis10 is looking for a Business Analyst fo...,Telecommuting not available|Travel not required,"Analysis, Analyst, Application, Business Analy...",0,1,0,0,2017
34,"VanderHouwen & Associates, Inc.","Contract W2, Contract",OR,Business Systems Analyst,VanderHouwen has more jobs you may like! Find ...,Telecommuting not available|Travel not required,"AI, Maya, Browzwear, 3D",0,0,1,0,2017
42,Amazon,"Full Time, Fulltime",TX,"Software Development Engineer, Big Data",Do you want to help build a highly personalize...,Telecommuting not available|Travel not required,DESCRIPTION Do you want to help build a highly...,0,1,0,0,2017


In [18]:
main.to_csv('2017_job_data.csv')

In [19]:
main.shape

(3289, 12)